In [ ]:
# mmcv 설치하기
!pip3 install openmim
!mim install mmcv-full

In [ ]:
# mmdetection git clone
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

In [ ]:
import mmdet
import mmcv
print(mmdet.__version__)

2.28.2


/usr/local/lib/python3.9/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
# SMD Dataset Registry
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset

@DATASETS.register_module(force=True)
class SMDDataset(CocoDataset):
  CLASSES = ('Ferry','Buoy', 'Vessel/ship', 'Speed boat','Boat','Kayak','Sail boat','Swimming person','Flying bird/plane','Other') # 이렇게 CLASSES변수만 지정해주면 된다.

YOLOx Training

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# config file setting
config_file = '/content/mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py'

In [ ]:
# YOLOX pretrained weight Model Download
!mkdir /content/mmdetection/checkpoints
!wget -O /content/mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth

In [ ]:
# Load YOLOv3 Config Object
from mmcv import Config
%cd ./mmdetection
cfg = Config.fromfile(config_file)
%cd ..

/content/mmdetection
/content


In [ ]:
# Modify dataset type and path.
cfg.dataset_type = 'SMDDataset'
cfg.data_root = '/content/drive/MyDrive/SMDDataset/35_Frame'

cfg.train_dataset.dataset.type = 'SMDDataset'
cfg.train_dataset.dataset.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/train.json'
cfg.train_dataset.dataset.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/train/'


cfg.data.train.dataset.type = 'SMDDataset'
cfg.data.train.dataset.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.train.dataset.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'


cfg.data.val.type = 'SMDDataset'
cfg.data.val.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.val.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'

cfg.data.test.type = 'SMDDataset'
cfg.data.test.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.test.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'

# Batch size (samples per GPU).
cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

# Modify number of classes as per the model head.
cfg.model.bbox_head.num_classes = 10

# model checkpoint path. 
cfg.load_from = '/content/mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'

# Learning Rate Setting
cfg.optimizer.lr = 0.0001
cfg.lr_config.min_lr_ratio = 0.01
cfg.log_config.interval = 50 # 몇 개 사진을 train할때마다 log를 띄울 것인가

In [ ]:
from mmdet.apis import set_random_seed
# The output directory for training. As per the model name.
cfg.work_dir = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx'

# Evaluation Metric.
cfg.evaluation.metric = 'bbox'
cfg.evaluation.save_best = 'auto'

# Evaluation times.
cfg.evaluation.interval = 5

# Checkpoint storage interval.
cfg.checkpoint_config.interval = 70

# Set random seed for reproducible results.
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda' 

cfg.runner.max_epochs = 210 # 총 몇 epoch를 훈련할 것인가

# Use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

print('#'*50)
print(f'Config:\n{cfg.pretty_text}')

##################################################
Config:
optimizer = dict(
    type='SGD',
    lr=0.0001,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup='exp',
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.01)
runner = dict(type='EpochBasedRunner', max_epochs=210)
checkpoint_config = dict(interval=70)
log_config = dict(
    interval=50,
    hooks=[dict(type='TextLoggerHook'),
           dict(type='TensorboardLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import os.path as osp
import mmcv
# Build dataset
datasets = [build_dataset(cfg.data.train)]
# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [ ]:
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Start Training
train_detector(model, datasets, cfg, distributed=False, validate=True)

여기서부터는 **Inference code**입니다

In [ ]:
from mmdet.apis import inference_detector
from mmdet.apis import init_detector
from mmcv import Config
import argparse
import mmcv
import glob as glob
import os

# load config, weights
config_file = '/content/mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py'
checkpoint_file = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx/YOLOX_final.pth'
cfg = Config.fromfile(config_file)

# video file path for inference
video_inputs = [\
    "/content/drive/MyDrive/test_images/test_video.mp4"]

video_outputs = [\
    "/content/drive/MyDrive/Model_Weight(SMD)/YOLOx/result/test_video_result.mp4"]

# confidence-theshold for inference
threshold = 0.5

In [ ]:
# Modify dataset type and path.
cfg.dataset_type = 'SMDDataset'
cfg.data_root = '/content/drive/MyDrive/SMDDataset/35_Frame'

cfg.train_dataset.dataset.type = 'SMDDataset'
cfg.train_dataset.dataset.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/train.json'
cfg.train_dataset.dataset.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/train/'


cfg.data.train.dataset.type = 'SMDDataset'
cfg.data.train.dataset.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.train.dataset.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'


cfg.data.val.type = 'SMDDataset'
cfg.data.val.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.val.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'

cfg.data.test.type = 'SMDDataset'
cfg.data.test.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.test.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'

# Batch size (samples per GPU).
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 2

# Modify number of classes as per the model head.
cfg.model.bbox_head.num_classes = 10

# model checkpoint path. 
cfg.load_from = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx/YOLOX_final.pth'

from mmdet.apis import set_random_seed

# The output directory for training. As per the model name.
cfg.work_dir = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx'

# Set random seed for reproducible results.
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda' 

# Use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]
# Build the model.
model = init_detector(cfg, checkpoint_file)

load checkpoint from local path: /content/drive/MyDrive/Model_Weight(SMD)/YOLOx/YOLOX_final.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, ema_backbone_stage1_1_main_conv_bn_running_var, ema_backbone_stage1_1_main_conv_bn_num_batches_tracked, ema_backbone_stage1_1_short_conv_conv_weight, ema_backbone_stage1_1_short_conv_bn

In [ ]:
# Start Image file inference

# Classes for SMD Dataset 
labels_to_names_seq = {0:'Ferry',1:'Buoy',2:'Vessel/Ship',3:'Speed boat',4:'Boat',5:'Kayak',6:'Sail boat',7:'Swimming person',8:'Flying bird/plane',9:'Other'}

labels_to_names = {0:'Ferry',1:'Buoy',2:'Vessel/Ship',3:'Speed boat',4:'Boat',5:'Kayak',6:'Sail boat',7:'Swimming person',8:'Flying bird/plane',9:'Other'}

# Function for visualization of inference result
def get_detected_img(model, img_array,  score_threshold=0.45, is_print=True):
  # 인자로 들어온 image_array를 복사. 
  draw_img = img_array.copy()
  bbox_color=(0, 255, 0) # bbox는 Green색으로 표시한다
  text_color=(0, 0, 255) # text는 Red색으로 표시한다

  results = inference_detector(model, img_array)


  for result_ind, result in enumerate(results):
    if len(result) == 0:
      continue
     
    result_filtered = result[np.where(result[:, 4] > score_threshold)]
    
    # confidence score가 threshold 이상으로 탐지된 object에 대하여 표시한다
    # 해당 클래스 별로 Detect된 여러개의 오브젝트 정보가 2차원 array에 담겨 있으며, 이 2차원 array를 row수만큼 iteration해서 개별 오브젝트의 좌표값 추출. 
    for i in range(len(result_filtered)):
      # 좌상단, 우하단 좌표 추출. 
      left = int(result_filtered[i, 0])
      top = int(result_filtered[i, 1])
      right = int(result_filtered[i, 2])
      bottom = int(result_filtered[i, 3])
      caption = "{}: {:.4f}".format(labels_to_names_seq[result_ind], result_filtered[i, 4]) # Bounding Box위에 표시할 caption
      cv2.rectangle(draw_img, (left, top), (right, bottom), color=bbox_color, thickness=3)
      cv2.putText(draw_img, caption, (int(left), int(top - 7)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
      if is_print:
        print(caption)

  return draw_img #  bounding box와 caption이 표시된 image를 반환한다

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

input_images = [file for file in os.listdir("/content/drive/MyDrive/test_images") if file.split(".")[1] == "jpg"]
# JPG 또는 jpg파일만 inference시행
for input_image in input_images:
  output_image = "/content/drive/MyDrive/Model_Weight(SMD)/YOLOx/result/" + input_image[:-5] + "_result.jpg"
  input_image = "/content/drive/MyDrive/test_images/" + input_image
  img_arr = cv2.imread(input_image)
  detected_img = get_detected_img(model, img_arr,  score_threshold=0.45, is_print=False)
  cv2.imwrite(output_image, detected_img)

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# Start Video Inference
import argparse
import mmcv
import time
import cv2

for i in range(len(video_inputs)):
  input_video = video_inputs[i]
  output_video = video_outputs[i]

  cap = mmcv.VideoReader(input_video)
  save_name = output_video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  
  out = cv2.VideoWriter(
      save_name, fourcc, cap.fps,
      (cap.width, cap.height)
  )
  frame_count = 0 # To count total frames.
  total_fps = 0 # To get the final frames per second.

  for frame in mmcv.track_iter_progress(cap):
      # Increment frame count.
      frame_count += 1
      start_time = time.time()# Forward pass start time.
      result = inference_detector(model, frame)
      end_time = time.time() # Forward pass end time.
      # Get the fps.
      fps = 1 / (end_time - start_time)
      # Add fps to total fps.
      total_fps += fps
      show_result = model.show_result(frame, result, score_thr=threshold)
      # Write the FPS on the current frame.
      cv2.putText(
          show_result, f"{fps:.3f} FPS", (15, 30), cv2.FONT_HERSHEY_SIMPLEX,
          1, (0, 0, 255), 2, cv2.LINE_AA
      )
      # mmcv.imshow(show_result, 'Result', wait_time=1)
      out.write(show_result)
  # Release VideoCapture()
  out.release()
  # Close all frames and video windows
  cv2.destroyAllWindows()
  # Calculate and print the average FPS
  avg_fps = total_fps / frame_count
  print(f"Average FPS: {avg_fps:.3f}")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 184/184, 4.0 task/s, elapsed: 46s, ETA:     0s
Average FPS: 29.917


***Performance Evaluation***

아래 셀을 수행하기 전 런타임 재시작을 하지 않으면 버그 발생 (Key error)

In [ ]:
### 아래는 런타임 다시 시작 후 실행
from mmcv import Config
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset
from mmdet.apis import set_random_seed

# Dataset re-registry
@DATASETS.register_module(force=True)
class SMDDataset(CocoDataset):
  CLASSES = ('Ferry','Buoy', 'Vessel/ship', 'Speed boat','Boat','Kayak','Sail boat','Swimming person','Flying bird/plane','Other')

# Reset config and rebuild model
# Modify dataset type and path.
config_file = '/content/mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py'
checkpoint_file = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx/YOLOX_final.pth'
cfg = Config.fromfile(config_file)


# Modify dataset type and path.
cfg.dataset_type = 'SMDDataset'
cfg.data_root = '/content/drive/MyDrive/SMDDataset/35_Frame'

cfg.train_dataset.dataset.type = 'SMDDataset'
cfg.train_dataset.dataset.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/train.json'
cfg.train_dataset.dataset.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/train/'


cfg.data.train.dataset.type = 'SMDDataset'
cfg.data.train.dataset.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.train.dataset.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'


cfg.data.val.type = 'SMDDataset'
cfg.data.val.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.val.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'

cfg.data.test.type = 'SMDDataset'
cfg.data.test.ann_file = '/content/drive/MyDrive/SMDDataset/35_Frame/val.json'
cfg.data.test.img_prefix = '/content/drive/MyDrive/SMDDataset/35_Frame/val/'

# Batch size (samples per GPU).
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 2

# Modify number of classes as per the model head.
cfg.model.bbox_head.num_classes = 10

# model checkpoint path. 
cfg.load_from = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx/YOLOX_final.pth'


from mmdet.apis import set_random_seed
# The output directory for training. As per the model name.
cfg.work_dir = '/content/drive/MyDrive/Model_Weight(SMD)/YOLOx'

# Evaluation Metric.
cfg.evaluation.metric = 'bbox'
cfg.evaluation.save_best = 'auto'

# Set random seed for reproducible results.
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda' 


# Use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

/usr/local/lib/python3.9/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
cfg.evaluation.classwise = True

In [ ]:
from mmdet.datasets import (build_dataloader, build_dataset,replace_ImageToTensor)

# test용 Dataset과 DataLoader 생성. 
# build_dataset()호출 시 list로 감싸지 않는 것이 train용 dataset 생성시와 차이
# train용 dataset는 list로 감싸야 하고, test용 dataset는 list로 감싸지 않는다
dataset = build_dataset(cfg.data.test) # test dataset을 만든다
data_loader = build_dataloader(
        dataset,
        # 반드시 아래 samples_per_gpu 인자값은 1로 설정
        samples_per_gpu=cfg.data.samples_per_gpu, # 그래서 아까 config설정할 때 이 값을 1로 설정한 것이다
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

# 
next(iter(data_loader))

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# checkpoint 저장된 model 파일을 이용하여 모델을 생성, 이때 Config는 위에서 update된 config 사용. 
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')

load checkpoint from local path: /content/drive/MyDrive/Model_Weight(SMD)/YOLOx/YOLOX_final.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, ema_backbone_stage1_1_main_conv_bn_running_var, ema_backbone_stage1_1_main_conv_bn_num_batches_tracked, ema_backbone_stage1_1_short_conv_conv_weight, ema_backbone_stage1_1_short_conv_bn

In [ ]:
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

model_ckpt = MMDataParallel(model_ckpt, device_ids=[0]) # 병렬처리를 하기 위함
# single_gpu_test() 를 호출하여 test데이터 세트의 interence 수행. 반드시 batch size는 1이 되어야 함. 
# 위에서 만든 /content/show_test_output 디렉토리에 interence 결과가 시각화된 이미지가 저장됨.
# 이것도 batch_size=1이 아니면 문제가 생긴다.
# test dataset의 결과(inference가 적용된 이미지 결과)를 저장
outputs = single_gpu_test(model_ckpt, data_loader, True, '/content', 0.3)

[                                                  ] 0/113, elapsed: 0s, ETA:

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 113/113, 2.1 task/s, elapsed: 53s, ETA:     0s

In [ ]:
# evaluation실시
metric = dataset.evaluate(outputs, metric='bbox', classwise = True)
print(metric)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.09s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.705
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.949
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.764
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.509
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.748
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.862
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.740
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.740
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.740
 Average Recall     (AR) @[ IoU=0.50:0.95